## **Paradigma funcional en Python**

#### **Funciones de Orden Superior**

Aquellas que reciben una o más funciones como argumento o que devuelven una función como retorno.

In [ ]:
'''
Veamos este ejemplo en python de una función de orden superior que aplica una función a cada elemento de una lista.
'''

from typing import TypeVar, Union
from collections.abc import Callable, Sequence

Numerico = Union[int, float]

T = TypeVar("T")

def aplicar_operacion(lista: Sequence[T], operacion: Callable[[T], T]) -> Sequence[T]:
    resultado = []
    for elemento in lista:
        resultado.append(operacion(elemento))
    return resultado

# Definición de funciones que se aplicarán a la lista
def cuadrado(x: Numerico) -> Numerico:
    return x * x

def inverso(x: Numerico) -> Numerico:
    return 0 - x

# Uso de funcion de orden superior
numeros: list[int] = [1, -2, 3, -4, 5, -6]
numeros_cuadrados = aplicar_operacion(numeros, cuadrado)  # Elevar al cuadrado
numeros_inversos = aplicar_operacion(numeros, inverso)   # Inverso aditivo

print(numeros_cuadrados)  # [1, 4, 9, 16, 25, 36]
print(numeros_inversos)  # [-1, 2, -3, 4, -5, 6]

#### **Composición de funciones**
Combinar 2 o más funciones de manera que la salida de una f se convierte en la entrada de la segunda, y así sucesivamente

In [ ]:
# VERSIÓN IMPERATIVA

def add_elemento(xs: list[int], x: int) -> None:
    xs.append(x)

lista_enteros: list[int] = []
add_elemento(lista_enteros, 1)
add_elemento(lista_enteros, 2)
add_elemento(lista_enteros, 3)
print(lista_enteros)

In [ ]:
# VERSIÓN FUNCIONAL

def add_elemento(xs: list[int], x: int) -> list[int]:
    ys: list[int] = xs.copy()
    ys.append(x)
    return ys

lista_enteros: list[int] = add_elemento(add_elemento(add_elemento([], 1), 2), 3)
print(lista_enteros)

''' El cambio más importante es devolver la estructura modificada en la función, de forma que pueda ser consumida sucesivamente 
como argumento de la próxima. '''


### **Inmutabilidad**

##### **Transitividad**
Recordar que debemos verificar que los atributos de un objeto inmutable sean también inmutables, o de lo contrario contemplar en que no puedan mutar.

In [ ]:
from typing import TypeVar, Generic

T = TypeVar("T")

class ContenedorInmutable(Generic[T]):
    def __init__(self, valor: T):
        self._valor: T = valor
    
    def contenido(self) -> T:
        return self._valor

xs: list[int] = [1, 2, 3]
contenedor: ContenedorInmutable[list[int]] = ContenedorInmutable(xs)
xs[0] = 9

print(contenedor.contenido())   # [9, 2, 3]

#### **Clases inmutables: Properties**
Sabemos que en Python existe posibilidad de convertir atributos en propiedades para mejorar el encapsulamiento de la clase. Una estrategia sería convertir los atributos en propiedades de sólo lectura, es decir, no definir los setters.

In [ ]:
class MiClaseInmutable:
    def __init__(self, valor_inicial):
        self._valor = valor_inicial
    
    @property
    def valor(self):
        return self._valor

objeto_inmutable = MiClaseInmutable(20)
objeto_inmutable.valor                      # 20
objeto_inmutable.valor = 10                 # AttributeError: property 'valor' of 'MiClaseInmutable' object has no setter
objeto_inmutable._valor = 10                # Haciendo esto si que modifica el valor
objeto_inmutable.valor                      # 10

#### **Clases imnutables: Métodos especiales '__setattr__' y '__delattr__'**

In [ ]:
class MiClaseInmutable:
    __slots__ = ('_valor',)

    def __init__(self, valor_inicial):
        super().__setattr__('_valor', valor_inicial)
    
    def __setattr__(self, __name: str, __value: any) -> None:
        raise AttributeError(f'No es posible setear el atributo {__name}')
    
    def __delattr__(self, __name: str) -> None:
        raise AttributeError(f'No es posible eliminar el atributo {__name}')
    
    def valor(self):
        return self._valor
    

'''
En la inicialización debemos utilizar el super().__setattr__() porque el propio devuelve una excepción. 
Entonces una vez inicializado el objeto, nunca podremos modificarlo.
'''

#### **Clases imnutables: Named Tuples**

In [16]:
from collections import namedtuple

MiClaseInmutable = namedtuple('MiClaseInmutable', 'valor1 valor2')
mi_obj = MiClaseInmutable(10, 20)
mi_obj                  # MiClaseInmutable(valor1=10, valor2=20)
mi_obj.valor1           # 10
mi_obj.valor2           # 20

MiClaseInmutable(valor1=10, valor2=20)


El problema con esta estrategia es que perdemos el concepto de encapsulamiento que nos proveen las clases, vinculando la estructura con el comportamiento. Como solución podemos definir nuestra clase heredando desde *namedtuple*

In [ ]:
from collections import namedtuple

class MiClaseInmutable(namedtuple('MiClaseInmutable', 'valor1 valor2')):
    __slots__ = ()
    def __repr__(self) -> str:
        return f'{super().__repr__()} INMUTABLE'
    
mi_obj = MiClaseInmutable(10, 20)
mi_obj   # MiClaseInmutable(valor1=10, valor2=20) INMUTABLE

'''
Debemos agregar __slots__ para evitar que la clase pueda aceptar nuevos atributos, 
pero luego podemos definir el comportamiento que deseemos, como en el ejemplo sobreescribiendo el método especial __repr__.'''

#### **Clases imnutables: dataclasses**

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True) # En True este parámetro evita la asignacion de nuevos valores
class Persona:
    nombre: str
    apellido: str
    edad: int

    def es_adulta(self):
        return self.edad >= 18
    
p = Persona("Julia", "Martinez", 22)
print(p)        # Persona(nombre='Julia', apellido='Martinez', edad=22)
p.edad = 20     # FrozenInstanceError: cannot assign to field 'edad'

##### **Transparencia referencial**

Podríamos reemplazar a cierta expresión de una función y argumentos aplicados simplemente con su valor de retorno y así no se producirían cambios semánticos en el programa, como por ejemplo:

In [ ]:
def suma(x: int, y: int) -> int:
    return x + y

nro: int = suma(10, 6) * 2
nro: int = 16 * 2           # Reemplazamos suma(10, 6) por su valor evaluado 16

Ejemplo de efecto secundario al ejecutar un programa, donde la función *duplicar_elemento* produce de forma sutil un efecto secundario al modificar los elementos de la lista original que se pasa como argumento.

In [ ]:
def duplicar_elemento(lista: list[int], indice: int) -> list[int]:
    lista[indice] *= 2
    return lista

def duplicar_elemento_pura(lista: list[int], indice: int) -> list[int]:
    nueva_lista = lista.copy()
    nueva_lista[indice] *= 2
    return nueva_lista

# Uso de ambas funciones
original: list[int] = [1, 2, 3]
resultado1: list[int] = duplicar_elemento(original, 1)
resultado2: list[int] = duplicar_elemento_pura(original, 1)

print(f"Impura: {resultado1}")  # Salida: [1, 4, 3]
print(f"Pura: {resultado2}")  # Salida: [1, 8, 3]

#### **Evaluación perezosa**

Establece que la evaluación de una expresión puede dilatarse hasta que sea necesario su valor.
A su vez permite trabajar con estructuras infinitas, mediante la implementación de funciones generadoras, las cuales retornan un iterador perezoso con la palabra reservada yield.

In [ ]:
# Ejemplo con yield, usado para trabajar estructuras infinitas donde se devuelve un iterador perezoso

from collections.abc import Iterator

def genera_saludo() -> Iterator[str]:
    yield "Hola"
    yield "Buenas"
    yield "Buen día"

iterador_saludos = genera_saludo()
print(next(iterador_saludos))   # Hola
print(next(iterador_saludos))   # Buenas
print(next(iterador_saludos))   # Buen día
print(next(iterador_saludos))   # Error StopIteration

In [ ]:
for saludo in genera_saludo():
    print(saludo)

Mediante el uso del funciones o expresiones generadoras podemos lograr en Python una evaluación diferida al momento en el cual se lo necesita (se consume el valor retornado). Esto es útil para modelar estructuras infinitas, veamos con un ejemplo.

In [29]:
from collections.abc import Iterator

def positivos_pares() -> Iterator[int]:
    numero: int = 0
    while True:
        yield numero
        numero += 2

# Podemos implementarlo con una expresión generadora usando compresión de listas

positivos_pares = (x for x in range(0, 10, 2))  # <generator object <genexpr> ...>

### **Transformación de funciones**

#### **Currificación**
Conversión de una función con n argumentos en n funciones con un único argumento. 


In [ ]:
# Función simple de suma
def suma(x, y):
    return x + y

# Función currificada de suma
def suma_curry(x):
    def suma_x(y):
        return x + y
    return suma_x

print(suma(1, 3))
print(suma_curry(1)(3))

Otra opción más simple sería devolver directamente una expresión *lambda*

In [ ]:
def suma_curry(x):
    return lambda y: x + y

También podemos aprovechar la aplicación parcial para definir nuevas funciones.

In [33]:
def doble(x):
    return suma_curry(x)(x)

def incrementar_10(x):
    return suma_curry(10)(x)

Si deseamos *currificar* una función deberíamos generar tantos niveles de anidado como parámetros tiene la función original.

In [ ]:
def suma_xyz(x):
    def suma_x(y):
        def suma_y(z):
            return x + y + z
        return suma_y
    return suma_x

suma_xyz(1)(2)(3)

*functools.partial*: del módulo functools, que nos permite realizar la vinculación de la aplicación parcial a otra función. Por lo cual puede resultar en una herramienta útil en lugar de currificar nuestras funciones.

In [ ]:
from functools import partial

def producto(x: int, y: int) -> int:
    return x * y

producto_10 = partial(producto, 10)
producto_10(2)  # 20

*pymonad.tools.curry*: el decorador curry() también puede ser útil para facilitar la currificación de una función, solo hay que pasarle la cantidad de argumentos con la que se currifica. 

In [ ]:
from pymonad.tools import curry

@curry(2)
def producto(x: int, y: int) -> int:
    return x * y

producto_10 = producto(10)
producto_10(2)   # 20

#### **Composición con decoradores**
Es otra opción para la composición de funciones, ya que un decorador básicamente realiza mi_funcion = decorador(mi_funcion).

Veamos ahora un ejemplo sencillo, podríamos definir un decorador para limpiar los espacios al principio y final de una cadena.

In [ ]:
from collections.abc import Callable
from functools import wraps

def trim(f: Callable[[str], str]) -> Callable[[str], str]:
    @wraps(f)
    def wrapper(texto: str) -> str:
        return f(texto).strip()
    return wrapper

@trim
def transforma_texto(texto: str) -> str:
    return texto.replace('.',' ')

transforma_texto('  esto es una prueba. ')  # 'esto es una prueba'

Podríamos extender la versión previa de forma que reciba parámetros que determinen si deseamos eliminar sólo espacios en el inicio o el final de la cadena.

In [40]:
from collections.abc import Callable
from functools import wraps

def trim(inicio: bool = True, fin: bool = True) -> Callable[[Callable[[str], str]], Callable[[str], str]]:
    def trim_deco(f: Callable[[str], str]) -> Callable[[str], str]:
        @wraps(f)
        def wrapper(texto: str) -> str:
            texto = f(texto)
            if inicio:
                texto = texto.lstrip()
            if fin:
                texto = texto.rstrip()
            return texto
        return wrapper
    return trim_deco

Ahora si aplicamos este nuevo decorador, debemos hacerlo con parámetros:



In [43]:
@trim(inicio=False)
def transforma_texto(texto: str) -> str:
    return texto.replace('.',' ')

transforma_texto('  esto es una prueba. ')  # '  esto es una prueba'

'  esto es una prueba'

In [42]:
@trim(fin=False)
def transforma_texto(texto: str) -> str:
    return texto.replace('.',' ')

transforma_texto('  esto es una prueba. ')  # 'esto es una prueba  '

'esto es una prueba  '

### **Iteración e iterables**

Veamos un ejemplo de cómo podríamos modelar la lógica de un for con enfoque funcional. Primero un ejemplo para calcular una potencia de 2:

In [45]:
def potencia2(n: int) -> int:
    retorno: int = 1
    for x in range(0, n):
        retorno *= 2
    return retorno

potencia2(11)   # 2048

2048

Veamos cómo podríamos resolver esto desde el paradigma funcional convirtiendo la estructura de iteración en una función pura.

In [46]:
def iterar(veces: int, func: Callable[..., any], valor: any) -> any:
    if veces <= 0:
        return valor
    else:
        return iterar(veces - 1, func, func(valor))
    
def potencia2(n: int) -> int:
    return iterar(n, lambda x: 2 * x, 1)

potencia2(11)   # 2048

2048

#### **Mapeo**
La función map() es de orden superior porque recibe otra como argumento.

In [49]:
xs: list[int] = [1, 2, 3, 4]
ys: list[int] = []
operacion = lambda x: x * x

cuadrados: map = map(operacion, xs)    # <map at 0x1beb3187940>, iterador perezoso
list(cuadrados)                        # [1, 4, 9, 16], recien con el list se aplica sobre cada elemento la operacion

[1, 4, 9, 16]

En este ejemplo podemos ver que el objeto retornado por map es un iterador perezoso, por lo cual recién cuando construimos una lista a partir de un iterador con el constructor list(), se itera sobre cada elemento para obtener su cuadrado.

In [50]:
# Otro ejemplo pasando más de un argumento
totales: list[int] = [100, 200, 300]
registros: list[int] = [50, 40, 120]
proporciones: map = map(lambda x, y: x / y, totales, registros)
list(proporciones)  # [2.0, 5.0, 2.5]

[2.0, 5.0, 2.5]

Cuando aplicamos map a listas es recomendable usar *list comprehensions*. 

In [51]:
# zip() -> permite construir un iterable de tuplas que contiene los elementos de cada iterable en el orden q los recibe.
proporciones: list[float] = [x/y for x,y in zip(totales, registros)]
proporciones # [2.0, 5.0, 2.5]

[2.0, 5.0, 2.5]

In [52]:
# Ejemplo de zip()
from collections.abc import Iterable

def mi_zip(*iterables: Iterable[any]) -> Iterator[tuple[any, ...]]:
    return map(lambda *elementos: tuple(elementos), *iterables)

list(mi_zip([1,2,3], ['a','b','c']))    # [(1, 'a'), (2, 'b'), (3, 'c')]

[(1, 'a'), (2, 'b'), (3, 'c')]

#### **Filtrado**
La función filter() es una función de orden superior, donde se aplica un predicado para generar una nueva colección con los elementos que cumplen la restricción.

Ejemplo de cómo podríamos filtrar una lista de números para obtener una lista con aquellos que son número par.

In [54]:
def es_par(n: int) -> bool:
    return n % 2 == 0

xs = [1, 2, 3, 4, 5, 6]
ys = []
filter(es_par, xs)  # <filter at 0x1d2af1aed70>, iterador perezoso
list(filter(es_par, xs))    # [2, 4, 6]

[2, 4, 6]

Imaginemos que necesitamos obtener una lista con los outliers de una muestra, para nuestro caso utilizaremos la técnica de detección mediante el cálculo de Z-score de cada observación y marcando aquellas que superan 3 desvíos de la media.

In [58]:
import numpy as np
from pymonad.tools import curry
from pymonad.reader import Compose

@curry(3)
def zscore(media: float, desvio: float, valor: float) -> float:
    return (valor - media) / desvio

def es_outlier(z_score: float, limite :float = 3) -> bool:
    return z_score > limite or z_score < (limite * -1)

# Generamos muestra random
muestra = np.random.normal(0, 5, 1000)
# Aplicamos parcialmente argumentos a zscore
zscore_muestra = zscore(muestra.mean(), muestra.std())
# Generamos nueva función predicado mediante la composición
filtro_outlier = Compose(zscore_muestra).then(es_outlier)

list(filter(filtro_outlier, muestra))   # lista con outliers

[15.38755886814691, -17.63251892076961, 16.615789044832425]

El siguiente ejemplo produce el mismo resultado que el código previo utilizando una expresión generadora.

In [59]:
[ x for x in muestra if es_outlier(zscore_muestra(x)) ]

[15.38755886814691, -17.63251892076961, 16.615789044832425]

#### **Reducción**
La función reduce() es una función de orden superior, donde se produce un valor a partir de la aplicación de una función acumuladora/combinadora/reductora sobre una estructura iterable.

In [63]:
from functools import reduce

def contar_letras(acumulado: int, elemento: str) -> int:
    return acumulado + len(elemento)

reduce(contar_letras, ['casa', 'puente', 'ojo'], 0)     # 13

13

valor_reducido = 0
    contar_letras(0, 'casa')            # valor_reducido: 4
        contar_letras(4, 'puente')      # valor_reducido: 10
            contar_letras(10, 'ojo')    # valor_reducido: 13

In [ ]:
from functools import reduce

xs = [3, 4, 1, 0, 11, 7, 5, 6]
# sum(xs)
reduce(lambda x, y: x + y, xs, 0)   # 37
# max(xs)
reduce(lambda x, y: x if x > y else y, xs)  # 11

La versión que realiza la suma de elementos utiliza 3 argumentos: **la función acumuladora que suma, la lista y el valor inicial**. La segunda versión que realiza la obtención del máximo utiliza 2 argumentos, **sólo la función acumuladora y la lista**.

 Lo importante aquí es comprender que el primer argumento **x corresponde al valor acumulado de la reducción**, mientras que el argumento **y es el elemento actual de la iteración**. Entonces el primer parámetro puede ser de otro tipo, pero siempre debe coincidir con el tipo de dato de retorno.